In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
import os
import contextily as cx
from shapely.geometry import Polygon, mapping
#from shapely.geometry import mapping
import rasterio
import rasterio.mask
from shapely.ops import unary_union
import salem
from mpl_toolkits.basemap import Basemap
#import earthpy.spatial as es

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

root = os.path.join('/', 'Users', 'jpcohen', 'EDS-220', 'thomas-fire-aftermath-and-BII')

az_fp = os.path.join(root, 'data', 'tl_2021_04_cousub', 'tl_2021_04_cousub.shp')

az = gpd.read_file(az_fp)

phoenix = az[az['NAME'] == 'Phoenix']